In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, models
from timm import create_model
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd 
import sys
import os

c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'datasets')))

In [3]:
sys.path

['C:\\Program Files\\Python312\\python312.zip',
 'C:\\Program Files\\Python312\\DLLs',
 'C:\\Program Files\\Python312\\Lib',
 'C:\\Program Files\\Python312',
 'c:\\Users\\tsfai\\Projects\\data-slayer-2.0\\venv',
 '',
 'c:\\Users\\tsfai\\Projects\\data-slayer-2.0\\venv\\Lib\\site-packages',
 'c:\\Users\\tsfai\\Projects\\data-slayer-2.0\\venv\\Lib\\site-packages\\win32',
 'c:\\Users\\tsfai\\Projects\\data-slayer-2.0\\venv\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\tsfai\\Projects\\data-slayer-2.0\\venv\\Lib\\site-packages\\Pythonwin',
 'c:\\Users\\tsfai\\Projects\\data-slayer-2.0\\datasets']

In [4]:
from custom_dataset import CustomImageDataset

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [31]:
def get_model(model_name):
    if model_name == "resnet18":
        model = models.resnet18(pretrained=True)
        num_features = model.fc.in_features
        model.fc = nn.Sequential(
            nn.Linear(num_features, 1),
            nn.Sigmoid()
        )
    elif model_name == "efficientnet":
        model = create_model('efficientnet_b0', pretrained=True)
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, 1),
            nn.Sigmoid()
        )
    elif model_name == "mobilenetv2":
        model = create_model('mobilenetv2_100', pretrained=True)
        num_features = model.classifier.in_features
        model.classifier = nn.Linear(num_features, 1)
    elif model_name == 'vit':
        model = create_model('vit_base_patch16_224', pretrained=True)
        num_features = model.head.in_features
        model.head = nn.Linear(num_features, 1)
    else:
        raise ValueError(f"Unknown model '{model_name}'.")
    
    model = model.to(device)

    return model

In [32]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.unsqueeze(1).float()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total * 100
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}")
        
        if (epoch + 1) % 5 == 0:
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    labels = labels.unsqueeze(1).float()
                    outputs = model(inputs)
                    predicted = (outputs > 0.5).float()
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
            test_acc = correct / total * 100
            print(f"Test Accuracy: {test_acc:.2f}%")
        
        scheduler.step()

In [33]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [34]:
data_train = pd.read_csv(os.path.join("..", "data", "revalidated", "data_train_f_001.csv"))

train_data, val_data = train_test_split(data_train, test_size=0.2, random_state=42)

train_data_paths = train_data['path'].values.tolist()
val_data_paths = val_data['path'].values.tolist()

train_data_labels = train_data['label'].values.tolist()
val_data_labels = val_data['label'].values.tolist()

train_dataset = CustomImageDataset(image_paths=train_data_paths, labels=train_data_labels, transform=transform)
val_dataset = CustomImageDataset(image_paths=val_data_paths, labels=val_data_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [35]:
models_to_try = ['resnet18', 'efficientnet', 'mobilenetv2', 'vit']

In [36]:
num_epochs = 10
criterion = nn.BCELoss()
learning_rate = 0.0001

In [37]:
models_trained = {model_name: None for model_name in models_to_try}
models_trained

{'resnet18': None, 'efficientnet': None, 'mobilenetv2': None, 'vit': None}

In [15]:
for model_name in models_to_try:
    print(f"Training model: {model_name}")
    models_trained[model_name] = get_model(model_name)

    optimizer = optim.Adam(models_trained[model_name].parameters(), lr=learning_rate)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    train_model(models_trained[model_name], train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=num_epochs)

Training model: resnet18


c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.0794, Accuracy: 97.26
Epoch 2/10, Loss: 0.0311, Accuracy: 98.84
Epoch 3/10, Loss: 0.0153, Accuracy: 99.39
Epoch 4/10, Loss: 0.0081, Accuracy: 99.77
Epoch 5/10, Loss: 0.0152, Accuracy: 99.51
Test Accuracy: 98.95%
Epoch 6/10, Loss: 0.0052, Accuracy: 99.80
Epoch 7/10, Loss: 0.0029, Accuracy: 99.91
Epoch 8/10, Loss: 0.0022, Accuracy: 99.97
Epoch 9/10, Loss: 0.0020, Accuracy: 99.91
Epoch 10/10, Loss: 0.0024, Accuracy: 99.94
Test Accuracy: 98.95%
Training model: efficientnet


c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tsfai\.cache\huggingface\hub\models--timm--efficientnet_b0.ra_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Epoch 1/10, Loss: 0.1292, Accuracy: 96.56
Epoch 2/10, Loss: 0.0308, Accuracy: 99.01
Epoch 3/10, Loss: 0.0192, Accuracy: 99.36
Epoch 4/10, Loss: 0.0118, Accuracy: 99.65
Epoch 5/10, Loss: 0.0099, Accuracy: 99.71
Test Accuracy: 98.72%
Epoch 6/10, Loss: 0.0044, Accuracy: 99.85
Epoch 7/10, Loss: 0.0042, Accuracy: 99.88
Epoch 8/10, Loss: 0.0028, Accuracy: 99.94
Epoch 9/10, Loss: 0.0023, Accuracy: 99.97
Epoch 10/10, Loss: 0.0033, Accuracy: 99.91
Test Accuracy: 98.95%
Training model: mobilenetv2


c:\Users\tsfai\Projects\data-slayer-2.0\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tsfai\.cache\huggingface\hub\models--timm--mobilenetv2_100.ra_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


TypeError: 'Linear' object is not subscriptable

In [17]:
torch.save(models_trained[models_to_try[0]].state_dict(), "../weights/weights_005_resnet.pth")
torch.save(models_trained[models_to_try[1]].state_dict(), "../weights/weights_005_efficientnet.pth")

In [38]:
for model_name in models_to_try[2:]:
    print(f"Training model: {model_name}")
    models_trained[model_name] = get_model(model_name)

    optimizer = optim.Adam(models_trained[model_name].parameters(), lr=learning_rate)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

    train_model(models_trained[model_name], train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=num_epochs)

Training model: mobilenetv2


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
